## Basic Usage

### Regression Data

The BKTR algorithm is only able to run on certain datasets respecting some criteria. This dataset should (most of the time) contain five different dataframes. 
  - A dataframe for the spatial covariates ($S$ x $c_s$)
  - A dataframe for the spatial point location coordinates ($S$ x $l_s$)
  - A dataframe for the temporal covariates ($T$ x $c_t$)
  - A dataframe for the temporal point location coordinates ($T$ x $l_t$)
  - A dataframe for the response variable ($S$ x $T$)

Where:
  - $S$ is the number of spatial points
  - $T$ is the number of temporal points
  - $l_s$ and $l_t$ are the number of dimensions used to represent the location of spatial and temporal points (respectively).
  - $c_s$ and $c_t$ are the number of covariates used for spatial and temporal points (respectively).

### Data Example

In this package we attached the same dataset as the one presented in the BKTR article. We can explore those datasets to be able to ensure that they fit the dimensions criteria.

In [ ]:
# TODO
# Clean data with the right final dimensions before
# Load all dataframes and show their dimensions

### Running BKTR
Once the dataframes have been loaded with the right dimensions, we can easily run the BKTR algorithm on our dataset.

In [ ]:
from pyBKTR.bktr import BKTRRegressor

bktr_regressor = BKTRRegressor(
    temporal_covariate_matrix=bixi_weather_matrix,
    spatial_covariate_matrix=bixi_station_matrix,
    y=bixi_y,
    omega=bixi_omega,
    rank_decomp=10,
    burn_in_iter=10,
    sampling_iter=5,
    spatial_kernel_x=spatial_kernel_x,
    temporal_kernel_x=temporal_kernel_x,
)
bktr_regressor.mcmc_sampling()

It is interesting to note that the package use default kernels and distance calculation methods that we think most users will want to use. By default, the `BKTRRegressor` use a Matern kernel $\frac{3}{2}$ and the haversine distance for the spatial coordinates, meaning that the `spatial_kernel_x` should have a dimension of $S$ x $2$ for (longitude, latitude). For temporal coordinates we use a SE Kernel by default with linear distance for the temporal coordinates meaning the `temporal_kernel_x` should have a dimension of $T$ x $1$.

Also, the `rank_decomp` parameter represent the rank of the decomposition used in the BKTR algorithm, in general a higher rank have the possibility to give more precise results at the cost of computation time. The `burn_in_iter` and `sampling_iter` params are used to dertermine the number of iterations of the algorithm, the `burn_in_iter` represent the number of iterations before we start the sampling (helps to the parameters to converge before starting the sampling phase) and the `sampling_iter` is the number of iterations used for sampling.

In the next section, we will demonstrate how to use different kernels and distance matrix to better fit your own data.